In [1]:
!pip install langchain
!pip install langchain-community
!pip install streamlit
!pip install PyPDF2
!pip install tqdm
!pip install opensearch-py
!pip install requests-aws4auth


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.0/109.0 kB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 8.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 310.2/310.2 kB 48.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.4/124.4 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 56.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 84.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 11.4 MB/s eta 0:00:00
  Attempting uninstall: packaging
    Found existing installation: packaging 21.3
    Uninstalling packaging-21.3:
      Successfully uninstalled packaging-21.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This 

In [3]:
import boto3

client = boto3.client('opensearchserverless', region_name='us-west-2')  # Replace with your region

# List all collections (optional, to find the ID of your collection)
collections = client.list_collections()
for collection in collections['collectionSummaries']:
    print(collection)

# Get details of a specific collection (replace with your collection ID)
collection_id = 'idwmpwfyanezjezlby32'
response = client.batch_get_collection(
    ids=[collection_id]
)

print(response)

# List indexes within the collection
response = client.list_indexes(
    CollectionId=collection_id
)

print(response)

{'arn': 'arn:aws:aoss:us-west-2:734389953405:collection/idwmpwfyanezjezlby32', 'id': 'idwmpwfyanezjezlby32', 'name': 'gen-ai-collection', 'status': 'ACTIVE'}
{'collectionDetails': [{'arn': 'arn:aws:aoss:us-west-2:734389953405:collection/idwmpwfyanezjezlby32', 'collectionEndpoint': 'https://idwmpwfyanezjezlby32.us-west-2.aoss.amazonaws.com', 'createdDate': 1717497807758, 'dashboardEndpoint': 'https://idwmpwfyanezjezlby32.us-west-2.aoss.amazonaws.com/_dashboards', 'description': '', 'id': 'idwmpwfyanezjezlby32', 'kmsKeyArn': 'auto', 'lastModifiedDate': 1717498232949, 'name': 'gen-ai-collection', 'standbyReplicas': 'ENABLED', 'status': 'ACTIVE', 'type': 'VECTORSEARCH'}], 'collectionErrorDetails': [], 'ResponseMetadata': {'RequestId': '160eb5b2-8566-4cf6-9dd9-c271875bdc51', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': '160eb5b2-8566-4cf6-9dd9-c271875bdc51', 'date': 'Tue, 04 Jun 2024 14:49:30 GMT', 'content-type': 'application/x-amz-json-1.0', 'content-length': '522', 'connect

AttributeError: 'OpenSearchServiceServerless' object has no attribute 'list_indexes'

In [4]:
import json
import os
import sys
import boto3
import streamlit as st
import PyPDF2 

## We will be using cohere Embeddings Model To generate Embedding

from langchain_community.embeddings import BedrockEmbeddings
from langchain.llms.bedrock import Bedrock
from langchain.schema import HumanMessage, SystemMessage
## Data Ingestion

import numpy as np
from langchain.text_splitter import RecursiveCharacterTextSplitter

## LLm Models
from langchain.prompts import PromptTemplate
from langchain.chains import RetrievalQA

## Bedrock Clients
bedrock=boto3.client(service_name="bedrock-runtime")
bedrock_embeddings=BedrockEmbeddings(model_id="cohere.embed-english-v3",client=bedrock)
from langchain_community.chat_models import BedrockChat


from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
def get_claude_llm():
    ##create the Anthropic Model
    llm=Bedrock(model_id="ai21.j2-mid-v1",client=bedrock,
                model_kwargs={'maxTokens':512})


In [10]:
import boto3
import time

# Initialize S3 and Textract clients
s3_client = boto3.client('s3')
textract_client = boto3.client('textract')

# Define your S3 bucket and PDF file name
bucket_name = 'gen-ai-dummy'
file_name = 'amazonq-developer-ug (1).pdf'  
output_file_name = 'extracted_text.txt'  

# Function to check job status
def check_job_status(job_id):
    while True:
        response = textract_client.get_document_analysis(JobId=job_id)
        status = response['JobStatus']
        if status in ['SUCCEEDED', 'FAILED']:
            return response
        print('Waiting for job to complete...')
        time.sleep(5)

# Function to process Textract response and retrieve all results
def get_textract_results(job_id):
    pages = []
    response = textract_client.get_document_analysis(JobId=job_id)
    pages.append(response)
    next_token = response.get('NextToken')

    while next_token:
        response = textract_client.get_document_analysis(JobId=job_id, NextToken=next_token)
        pages.append(response)
        next_token = response.get('NextToken')
    
    return pages

# Function to extract text and tables from Textract results
def extract_text_and_tables_from_results(pages):
    text_lines = []
    tables = []

    for page in pages:
        blocks = page['Blocks']
        for block in blocks:
            if block['BlockType'] == 'LINE':
                text_lines.append(block['Text'])
            elif block['BlockType'] == 'TABLE':
                table = extract_table(blocks, block)
                tables.append(table)
    
    return text_lines, tables

# Function to extract table data from blocks
def extract_table(blocks, table_block):
    rows = {}
    for relationship in table_block.get('Relationships', []):
        if relationship['Type'] == 'CHILD':
            for child_id in relationship['Ids']:
                cell = next((b for b in blocks if b['Id'] == child_id), None)
                if cell:
                    row_index = cell['RowIndex']
                    col_index = cell['ColumnIndex']
                    text = cell.get('Text', '')
                    if row_index not in rows:
                        rows[row_index] = {}
                    rows[row_index][col_index] = text
    table = []
    for row_index in sorted(rows.keys()):
        row = [rows[row_index].get(col_index, '') for col_index in sorted(rows[row_index].keys())]
        table.append(row)
    return table

# Function to save text and tables to a file and upload to S3
def save_text_and_tables_to_file_and_s3(text_lines, tables, output_file_path, bucket_name, output_file_name):
    with open(output_file_path, "w") as f:
        for line in text_lines:
            f.write(line + '\n')
        for table in tables:
            f.write('\nTABLE:\n')
            for row in table:
                f.write('\t'.join(row) + '\n')
    
    # Upload the file to S3
    s3_client.upload_file(output_file_path, bucket_name, output_file_name)
    print(f"Extracted text and tables saved to S3 bucket {bucket_name} as {output_file_name}")

# Start Textract job
response = textract_client.start_document_analysis(
    DocumentLocation={
        'S3Object': {
            'Bucket': bucket_name,
            'Name': file_name
        }
    },
    FeatureTypes=['TABLES', 'FORMS']
)
job_id = response['JobId']
print(f"Started Textract job with ID: {job_id}")

# Check job status and retrieve all results
response = check_job_status(job_id)
if response['JobStatus'] == 'SUCCEEDED':
    pages = get_textract_results(job_id)
    text_lines, tables = extract_text_and_tables_from_results(pages)
    output_file_path = '/tmp/extracted_text.txt'
    save_text_and_tables_to_file_and_s3(text_lines, tables, output_file_path, bucket_name, output_file_name)
else:
    print(f"Textract job failed with message: {response['StatusMessage']}")


Started Textract job with ID: f10e0865404b5e122a4d4d21b01271a2e5bcf172d882ff35b618769f5067a6f3
Waiting for job to complete...
Waiting for job to complete...
Extracted text and tables saved to S3 bucket gen-ai-dummy as extracted_text.txt


In [5]:
# Initialize S3 client
s3_client = boto3.client('s3')

# Define your S3 bucket and file names
bucket_name = 'gen-ai-dummy'
input_file_name = 'extracted_text.txt'
output_prefix = 'chunks/'

# Download the extracted text file from S3
s3_client.download_file(bucket_name, input_file_name, '/tmp/extracted_text.txt')

# Read the extracted text
with open('/tmp/extracted_text.txt', 'r') as file:
    extracted_text = file.read()

# Initialize the text splitter with chunk size of 500 and overlap of 150
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=150
)

# Split the text into chunks
chunks = text_splitter.split_text(extracted_text)

In [6]:
chunks

['Amazon Q Developer\nUser Guide\nWhat is Amazon Q Developer?\nNote\nPowered by Amazon Bedrock: AWS implements automated abuse detection. Because\nAmazon Q Developer is built on Amazon Bedrock, users can take full advantage of the\ncontrols implemented in Amazon Bedrock to enforce safety, security, and the responsible\nuse of artificial intelligence (AI).\nAmazon Q Developer is a generative artificial intelligence (AI) powered conversational assistant',
 'use of artificial intelligence (AI).\nAmazon Q Developer is a generative artificial intelligence (AI) powered conversational assistant\nthat can help you understand, build, extend, and operate AWS applications. You can ask questions\nabout AWS architecture, your AWS resources, best practices, documentation, support, and more.\nAmazon Q is constantly updating its capabilities so your questions get the most contextually\nrelevant and actionable answers.',
 'Amazon Q is constantly updating its capabilities so your questions get the most 

In [5]:
from langchain_community.chat_models import BedrockChat
from langchain.schema import HumanMessage, SystemMessage


bedrock_chat = BedrockChat(model_id="anthropic.claude-3-opus-20240229-v1:0", client=bedrock)

def get_summary(chunk):
    prompt = [
        SystemMessage(content="You are a helpful assistant."),
        HumanMessage(content=f"Summarize the following text:\n\n{chunk}")
    ]
    response = bedrock_chat(prompt)
    print("Summary Response:", response)
    return response.content

def get_faq(chunk):
    prompt = [
        SystemMessage(content="You are a helpful assistant."),
        HumanMessage(content=f"Generate a Frequently asked questions for the following text minimum 3 questions needs to be generated:\n\n{chunk}")
    ]
    response = bedrock_chat(prompt)
    print("FAQ Response:", response)
    return response.content


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `BedrockChat` was deprecated in LangChain 0.0.34 and will be removed in 0.3. An updated version of the class exists in the langchain-aws package and should be used instead. To use it run `pip install -U langchain-aws` and import as `from langchain_aws import ChatBedrock`.
  warn_deprecated(


In [6]:

for chunk in chunks:
    summary_response = get_summary(chunk)
    #faq_response = get_faq(chunk)
    # Print the responses to inspect their structure
    print("Summary_Response:", summary_response)
    #print("FAQ Response:", faq_response)
    break  # Just process one chunk for inspection


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


Summary Response: content="The text appears to be a user guide for Amazon Q Developer, a product or service offered by Amazon Web Services (AWS). The guide includes a copyright notice indicating that the content is the intellectual property of Amazon Web Services, Inc. and/or its affiliates. The text also mentions that Amazon's trademarks and trade dress cannot be used in connection with non-Amazon products or services in a way that may cause confusion among customers." response_metadata={'model_id': 'anthropic.claude-3-opus-20240229-v1:0', 'usage': {'prompt_tokens': 123, 'completion_tokens': 91, 'total_tokens': 214}} id='run-6a8c1802-490a-4651-8ed1-7a01e904fd5c-0'
Summary_Response: The text appears to be a user guide for Amazon Q Developer, a product or service offered by Amazon Web Services (AWS). The guide includes a copyright notice indicating that the content is the intellectual property of Amazon Web Services, Inc. and/or its affiliates. The text also mentions that Amazon's trade

In [18]:
from langchain_community.chat_models import BedrockChat
from langchain.schema import HumanMessage, SystemMessage
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed
import time
import random
import boto3


bedrock = boto3.client(service_name="bedrock-runtime")
bedrock_embeddings = BedrockEmbeddings(model_id="cohere.embed-english-v3", client=bedrock)
bedrock_chat = BedrockChat(model_id="anthropic.claude-3-opus-20240229-v1:0", client=bedrock)

def get_summary(chunk):
    prompt = [
        SystemMessage(content="You are a helpful assistant."),
        HumanMessage(content=f"Summarize the following text:\n\n{chunk}")
    ]
    response = bedrock_chat(prompt)
    return response.content

def get_faq(chunk):
    prompt = [
        SystemMessage(content="You are a helpful assistant."),
        HumanMessage(content=f"Generate a Frequently asked questions for the following text minimum 3 questions needs to be generated:\n\n{chunk}")
    ]
    response = bedrock_chat(prompt)
    return response.content


def truncate_text(text, max_length=2048):
    return text[:max_length]


def process_chunk(chunk):
    retry_attempts = 5
    for attempt in range(retry_attempts):
        try:
            summary = get_summary(chunk)
            faq = get_faq(chunk)
            break
        except Exception as e:
            if "throttlingexception" in str(e).lower():
                sleep_time = (2 ** attempt) + random.uniform(0, 1)
                time.sleep(sleep_time)  
            else:
                raise e
    
    truncated_chunk = truncate_text(chunk)
    truncated_summary = truncate_text(summary)
    truncated_faq = truncate_text(faq)
    
    chunk_embedding = bedrock_embeddings.embed_documents([truncated_chunk])[0]
    summary_embedding = bedrock_embeddings.embed_documents([truncated_summary])[0]
    faq_embedding = bedrock_embeddings.embed_documents([truncated_faq])[0]
    
    return {
        'chunk_embedding': chunk_embedding,
        'summary_embedding': summary_embedding,
        'faq_embedding': faq_embedding,
        'summary': summary,
        'faq': faq
    }


chunk_embeddings = []
summary_embeddings = []
faq_embeddings = []
summaries = []
faqs = []

def rate_limited_process_chunk(chunk):
    time.sleep(0.1)  
    return process_chunk(chunk)

with ThreadPoolExecutor(max_workers=10) as executor:  
    futures = [executor.submit(rate_limited_process_chunk, chunk) for chunk in chunks]
    for future in tqdm(as_completed(futures), total=len(futures), desc="Processing Chunks"):
        result = future.result()
        chunk_embeddings.append(result['chunk_embedding'])
        summary_embeddings.append(result['summary_embedding'])
        faq_embeddings.append(result['faq_embedding'])
        summaries.append(result['summary'])
        faqs.append(result['faq'])


Processing Chunks:   0%|          | 0/33 [01:01<?, ?it/s]


KeyboardInterrupt: 

In [13]:
import boto3
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth
from tqdm import tqdm

# AWS credentials and region
session = boto3.Session()
credentials = session.get_credentials()
region = 'us-west-2'
service = 'es'

# Create AWSV4SignerAuth
awsauth = AWSV4SignerAuth(credentials, region)

# Initialize OpenSearch client
opensearch_client = OpenSearch(
    hosts=['https://om1w8c5015kig00im868.us-west-2.aoss.amazonaws.com'],
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection
)

index_name = 'genai-doc-index'

def create_document(chunk_embedding, summary_embedding, faq_embedding, chunk, summary, faq):
    return {
        'chunk_vector': chunk_embedding, 
        'summary_vector': summary_embedding,
        'faq_vector': faq_embedding,
        'chunk_text': chunk,
        'summary_text': summary,
        'faq_text': faq
    }

for i in tqdm(range(len(chunks)), desc="Indexing Documents"):
    chunk = chunks[i]
    summary = summaries[i]
    faq = faqs[i]
    
    chunk_embedding = chunk_embeddings[i]
    summary_embedding = summary_embeddings[i]
    faq_embedding = faq_embeddings[i]
    
    document = create_document(chunk_embedding, summary_embedding, faq_embedding, chunk, summary, faq)
    
    response = opensearch_client.index(
        index=index_name,
        body=document,
        id=i
    )
    print(f"Indexed document {i}: {response}")


Indexing Documents:   0%|          | 0/935 [00:00<?, ?it/s]


AuthorizationException: AuthorizationException(403, 'Forbidden')

In [19]:
import boto3
from langchain_community.embeddings import BedrockEmbeddings
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.notebook import tqdm
import time
import random
from opensearchpy import OpenSearch, RequestsHttpConnection

# Initialize Bedrock client for embeddings
bedrock = boto3.client(service_name="bedrock-runtime")
bedrock_embeddings = BedrockEmbeddings(model_id="cohere.embed-english-v3", client=bedrock)

# Initialize OpenSearch client
host = 'https://idwmpwfyanezjezlby32.us-west-2.aoss.amazonaws.com'
index_name = 'vector-index-doc'

# If using IAM roles for authentication within SageMaker, no need for explicit username and password
client = OpenSearch(
    hosts=[host],
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection
)

def truncate_text(text, max_length=2048):
    return text[:max_length]

def process_chunk(chunk):
    truncated_chunk = truncate_text(chunk)
    
    retry_attempts = 5
    for attempt in range(retry_attempts):
        try:
            chunk_embedding = bedrock_embeddings.embed_documents([truncated_chunk])[0]
            break
        except Exception as e:
            if "throttlingexception" in str(e).lower():
                sleep_time = (2 ** attempt) + random.uniform(0, 1)
                time.sleep(sleep_time)
            else:
                raise e
    
    return {
        'chunk_embedding': chunk_embedding,
        'chunk': chunk
    }

def rate_limited_process_chunk(chunk):
    time.sleep(0.1)  
    return process_chunk(chunk)

# List to store the embeddings
chunk_embeddings = []

# Sample chunks to process

# Process chunks and collect embeddings
with ThreadPoolExecutor(max_workers=10) as executor:  
    futures = [executor.submit(rate_limited_process_chunk, chunk) for chunk in chunks]
    for future in tqdm(as_completed(futures), total=len(futures), desc="Processing Chunks"):
        result = future.result()
        chunk_embeddings.append(result)

# Function to create documents with embeddings
def create_document(chunk, embedding):
    return {
        'chunk': chunk,
        'chunk_embedding': embedding
    }

# Step 4: Index Documents in OpenSearch
for i, result in enumerate(chunk_embeddings):
    document = create_document(result['chunk'], result['chunk_embedding'])
    response = client.index(
        index=index_name,
        body=document
    )
    print(f'Document {i} indexed with response: {response}')

print("All chunks have been indexed successfully.")


Processing Chunks:   0%|          | 0/33 [00:00<?, ?it/s]

AuthorizationException: AuthorizationException(403, 'Forbidden')

In [8]:
import boto3
from langchain_community.embeddings import BedrockEmbeddings
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm.notebook import tqdm
import time
import random
from opensearchpy import OpenSearch, RequestsHttpConnection, AWSV4SignerAuth

# Initialize Bedrock client for embeddings
bedrock = boto3.client(service_name="bedrock-runtime")
bedrock_embeddings = BedrockEmbeddings(model_id="cohere.embed-english-v3", client=bedrock)

# Initialize OpenSearch client with IAM authentication
region = 'us-west-2'  # e.g., 'us-west-2'
service = 'es'
credentials = boto3.Session().get_credentials()
awsauth = AWSV4SignerAuth(credentials, region, service)

# Use your cluster endpoint here
host = 'https://idwmpwfyanezjezlby32.us-west-2.aoss.amazonaws.com'
index_name = 'vector-index-doc'

client = OpenSearch(
    hosts=[host],
    http_auth=awsauth,
    use_ssl=True,
    verify_certs=True,
    connection_class=RequestsHttpConnection
)

def truncate_text(text, max_length=2048):
    return text[:max_length]

def process_chunk(chunk):
    truncated_chunk = truncate_text(chunk)
    
    retry_attempts = 5
    for attempt in range(retry_attempts):
        try:
            chunk_embedding = bedrock_embeddings.embed_documents([truncated_chunk])[0]
            break
        except Exception as e:
            if "throttlingexception" in str(e).lower():
                sleep_time = (2 ** attempt) + random.uniform(0, 1)
                time.sleep(sleep_time)
            else:
                raise e
    
    return {
        'chunk_embedding': chunk_embedding,
        'chunk': chunk
    }

def rate_limited_process_chunk(chunk):
    time.sleep(0.1)  
    return process_chunk(chunk)

# List to store the embeddings
chunk_embeddings = []

# Sample chunks to process
chunks = ["Your text chunk 1", "Your text chunk 2", "Your text chunk 3"]  # Replace with your actual chunks

# Process chunks and collect embeddings
with ThreadPoolExecutor(max_workers=10) as executor:  
    futures = [executor.submit(rate_limited_process_chunk, chunk) for chunk in chunks]
    for future in tqdm(as_completed(futures), total=len(futures), desc="Processing Chunks"):
        result = future.result()
        chunk_embeddings.append(result)

# Function to create documents with embeddings
def create_document(chunk, embedding):
    return {
        'chunk': chunk,
        'chunk_embedding': embedding
    }

# Step 4: Index Documents in OpenSearch
for i, result in enumerate(chunk_embeddings):
    document = create_document(result['chunk'], result['chunk_embedding'])
    response = client.index(
        index=index_name,
        body=document
    )
    print(f'Document {i} indexed with response: {response}')

print("All chunks have been indexed successfully.")


Processing Chunks:   0%|          | 0/3 [00:00<?, ?it/s]

AuthorizationException: AuthorizationException(403, 'Forbidden')

In [25]:
chunk_embeddings

[{'chunk_embedding': [-0.02142334,
   -0.06298828,
   -0.014076233,
   -0.0011444092,
   -0.06121826,
   -0.010246277,
   -0.009277344,
   0.032470703,
   0.023376465,
   0.04812622,
   -0.017303467,
   0.0014915466,
   0.011352539,
   -0.019882202,
   -0.013999939,
   -0.0065612793,
   0.020355225,
   -0.018051147,
   0.01940918,
   -0.0057640076,
   0.009483337,
   0.015731812,
   0.02420044,
   0.0075149536,
   0.0032997131,
   -0.0039978027,
   0.07879639,
   0.011642456,
   -0.0012741089,
   -0.0059013367,
   0.011894226,
   -0.052215576,
   0.03164673,
   -0.002368927,
   -0.01852417,
   -0.049072266,
   -0.018737793,
   0.0029716492,
   -0.019515991,
   0.006187439,
   0.03845215,
   -0.029785156,
   -0.016784668,
   -0.072265625,
   -0.03149414,
   0.013008118,
   0.012832642,
   -0.00016784668,
   0.02758789,
   -0.028945923,
   0.00062942505,
   0.049682617,
   0.00472641,
   0.009986877,
   -0.006866455,
   0.040161133,
   0.021408081,
   -0.035827637,
   0.02986145,
   -0.0